# pFBA simulation for the MT-GSM of Poplar tricocarpa
## Original Code: Juliana Simas Barbosa
## Edits/streamlining: Wheaton Schroeder
### Latest Version: 08/30/2023

#### Make library imports

In [1]:
# Import necessary libraries:
import cobra
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import warnings

#### Define necessary functions

#### set information for the runs

In [2]:
# initial inputs:
## mass (gDW) of each tissue in year one 
### Leaf, stem, root
#note that these values will be the same for colntorl and drought, as prolonged drought introduced in the growing season modeled

#initial masses, from a paper
Init_S_mass = 185
Init_R_mass = 112
Init_L_mass = 121
total_mass = Init_S_mass + Init_R_mass + Init_L_mass

#fitted starts, from fitting of growth curves
#Init_S_mass = 100.54
#Init_R_mass = 67.403
#Init_L_mass = 86.659
#total_mass = Init_S_mass + Init_R_mass + Init_L_mass

#growth rates for the control model, calculated by fitting an experimental function
cont_leaf_g = 0.001946356
cont_root_g = 0.002960933
cont_stem_g = 0.003555685

#mass fractions at first time point
L_fraction_0 = Init_L_mass/total_mass
R_fraction_0 = Init_R_mass/total_mass
S_fraction_0 = Init_S_mass/total_mass

#photon uptake limits
leaf_photon_uptake_ub = 443
stem_photon_uptake_ub = 443

#non-growth associated maintenance
NGAM = 23.07238

#drought model file
drought_file = "./iPotri2999D.xml"

#read in the limit for CO2 based on minimum CO2 in pFBA
co2_min_cont_f = open("co2_cont_leaf.txt")
co2_min_cont = float(co2_min_cont_f.read())
co2_min_cont_f.close()

#specify fraction of control for stomatal conductance (gs), stem growth, and root growth for H1
#estimated from Bogeat-Triboulot et al, Plant Physiology, 2007
s_g_frac_H2 = 0.04
r_g_frac_H2 = 0.93
a_frac_H2 = 0.80

print("success")

success


#### Read in the drought model

In [3]:
drought_model = cobra.io.read_sbml_model(drought_file)

print("success")

No objective coefficients in model. Unclear what should be optimized


success


#### update intertissue transports through common pools based on tissue fractions

In [4]:
# Load inter-tissue transport reaction identifiers
it_ids = open('intertissue_transport_ids.txt')
IT_rxnIDs = it_ids.read().splitlines()
it_ids.close()

# Load inter-tissue transport reaction equations
it_id_eqs = open('intertissue_transport_eqs.txt')
IT_rxnEQs = it_id_eqs.read().splitlines()
it_id_eqs.close()

rxn_list = [False]*len(IT_rxnIDs)

# Extract the cobra Reaction object corresponding to the inter-tissue transport reactions
for idx,rxnID in enumerate(IT_rxnIDs):

  #rxn_list becomes an list of reaction objects
  rxn_list[idx] = drought_model.reactions.get_by_id(rxnID) 

# Edit the reaction stoichiometric coefficients to account for tissue fractions at different time-points
for idx, rxns in enumerate(rxn_list):

  #for debugging
  #print(idx)
  #turns out idx is index of reaction in the temp

  temp_halves = IT_rxnEQs[idx].split(' <=> ')
  
  if 'R2CP1' in str(IT_rxnIDs[idx]):   
    rxns.build_reaction_from_string('1 '+temp_halves[0]+' <=> '+str(R_fraction_0)+' '+temp_halves[1])
  
  if 'S2CP1' in str(IT_rxnIDs[idx]):
    rxns.build_reaction_from_string('1 '+temp_halves[0]+' <=> '+str(S_fraction_0)+' '+temp_halves[1])
  
  if 'S2CP2' in str(IT_rxnIDs[idx]):
    rxns.build_reaction_from_string('1 '+temp_halves[0]+' <=> '+str(S_fraction_0)+' '+temp_halves[1])
  
  if 'L2CP2' in str(IT_rxnIDs[idx]):
    rxns.build_reaction_from_string('1 '+temp_halves[0]+' <=> '+str(L_fraction_0)+' '+temp_halves[1])
        
  #for debugging
  #print(rxns)
  
  # Add the reactions back to the model
  #note: this will throw a lot of "ignoring reaction '...' since it already exists" warnings. It does appear that this does update the stoichiometry (I checked the stoichiometry of RXN_L2CP2_SUCROSE DARK before and after) this code block and the stoichiometry is updated, the warning I think
  
  #update the reaction stoichiometry
  drought_model.add_reactions([rxns])

  #give bounds allowing flow of metabolites in either direction
  drought_model.reactions.get_by_id(rxns.id).lower_bound = -10000; drought_model.reactions.get_by_id(rxns.id).upper_bound = 10000

print("success")

Ignoring reaction 'RXN_R2CP1_CO2_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_WATER_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_ZN2_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_PI_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_CU2_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_MN2_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_MG2_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_PROTON_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_NITRATE_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_SULFATE_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_AMMONIUM_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_FE2_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_FE3_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_K_LIGHT' since it already exists.
Ignoring reaction 'RXN_R2CP1_NA_LIGHT' since it already exists.
Ignoring re

success


#### Add pysical/sensible constraints to the model

In [5]:
#diurnal or sensible restrictions on flow of metabolites across system boundary
## NIGHT restrictions
drought_model.reactions.RXN_E2L_CO2_DARK.lower_bound = -10000;drought_model.reactions.RXN_E2L_CO2_DARK.upper_bound = 0              #CO2 export from leaf (heterotrophic growth)
drought_model.reactions.RXN_E2L_LIGHT__L___DARK.lower_bound = 0;drought_model.reactions.RXN_E2L_LIGHT__L___DARK.upper_bound = 0     #no light uptake in leaf (heterotrophic growth)
drought_model.reactions.RXN_E2S_LIGHT__S___DARK.lower_bound = 0;drought_model.reactions.RXN_E2S_LIGHT__S___DARK.upper_bound = 0     #no light uptake in stem (heterotrophic growth)
drought_model.reactions.RXN_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p__L___DARK.upper_bound = 0                                       #turns off rubisco CO2 fixation to be safe
drought_model.reactions.RXN_E2S_CO2_DARK.lower_bound = 0;drought_model.reactions.RXN_E2S_CO2_DARK.upper_bound = 10000               #stem exports CO2 during night

## DAY restrictions
drought_model.reactions.RXN_E2L_CO2_LIGHT.lower_bound = 0;drought_model.reactions.RXN_E2L_CO2_LIGHT.upper_bound = 10000               #leaf uptakes CO2 during day (phototrophic metabolism)
drought_model.reactions.RXN_E2L_LIGHT__L___LIGHT.lower_bound = 0;drought_model.reactions.RXN_E2L_LIGHT__L___LIGHT.upper_bound = 10000   #leaf uptakes light during day
drought_model.reactions.RXN_E2S_LIGHT__S___LIGHT.lower_bound = 0;drought_model.reactions.RXN_E2S_LIGHT__S___LIGHT.upper_bound = stem_photon_uptake_ub   #stem can uptake some limited light during day
drought_model.reactions.RXN_E2S_CO2_LIGHT.lower_bound = 0;drought_model.reactions.RXN_E2S_CO2_LIGHT.upper_bound = 10000             #stem exports CO2 during day

#### Add constraints to the model, fo the drought we need 2-3, 12, 14-19, 21, 23-26

In [6]:
# mass balance constraint should be automatically enforced by COBRA (equation 2)
# reaction flux bound constraints should be automatically enforced by COBRA (equation 3)

# enforce NGAM values (equation 12)
#drought_model.reactions.RXN_ATPM_c__L___LIGHT.upper_bound = NGAM; drought_model.reactions.RXN_ATPM_c__L___LIGHT.lower_bound = NGAM
#drought_model.reactions.RXN_ATPM_c__S___LIGHT.upper_bound = NGAM; drought_model.reactions.RXN_ATPM_c__S___LIGHT.lower_bound = NGAM
#drought_model.reactions.RXN_ATPM_c__R___LIGHT.upper_bound = NGAM; drought_model.reactions.RXN_ATPM_c__R___LIGHT.lower_bound = NGAM

#drought_model.reactions.RXN_ATPM_c__L___DARK.upper_bound = NGAM; drought_model.reactions.RXN_ATPM_c__L___DARK.lower_bound = NGAM
#drought_model.reactions.RXN_ATPM_c__S___DARK.upper_bound = NGAM; drought_model.reactions.RXN_ATPM_c__S___DARK.lower_bound = NGAM
#drought_model.reactions.RXN_ATPM_c__R___DARK.upper_bound = NGAM; drought_model.reactions.RXN_ATPM_c__R___DARK.lower_bound = NGAM

#require that all ngam values are equal
ngam_equal_1_dark = drought_model.problem.Constraint(drought_model.reactions.RXN_ATPM_c__L___DARK.flux_expression - drought_model.reactions.RXN_ATPM_c__S___DARK.flux_expression, ub = 0, lb = 0)
ngam_equal_2_dark = drought_model.problem.Constraint(drought_model.reactions.RXN_ATPM_c__L___DARK.flux_expression - drought_model.reactions.RXN_ATPM_c__R___DARK.flux_expression, ub = 0, lb = 0)
ngam_equal_1_light = drought_model.problem.Constraint(drought_model.reactions.RXN_ATPM_c__L___LIGHT.flux_expression - drought_model.reactions.RXN_ATPM_c__S___LIGHT.flux_expression, ub = 0, lb = 0)
ngam_equal_2_light = drought_model.problem.Constraint(drought_model.reactions.RXN_ATPM_c__L___LIGHT.flux_expression - drought_model.reactions.RXN_ATPM_c__R___LIGHT.flux_expression, ub = 0, lb = 0)
ngam_equal_dark_light = drought_model.problem.Constraint(drought_model.reactions.RXN_ATPM_c__L___LIGHT.flux_expression - drought_model.reactions.RXN_ATPM_c__L___DARK.flux_expression, ub = 0, lb = 0)

drought_model.add_cons_vars(ngam_equal_1_dark)
drought_model.add_cons_vars(ngam_equal_2_dark)
drought_model.add_cons_vars(ngam_equal_1_light)
drought_model.add_cons_vars(ngam_equal_2_light)
drought_model.add_cons_vars(ngam_equal_dark_light)

## Imposes that the transport of CO2 from the stem xylem to leaves is at most 2.7% of absorbed atmospheric CO2 (equation 14)
#RXN_L2CP2_CO2_LIGHT is positive if loading CO2 into the CP2 
#RXN_E2L_CO2_LIGHT is positive if CO2 is moving into the leaf extracellular compartment from the shared extracellular compartment
co2tr = drought_model.problem.Constraint(drought_model.reactions.RXN_L2CP2_CO2_LIGHT.flux_expression + 0.027 * drought_model.reactions.RXN_E2L_CO2_LIGHT.flux_expression, ub = 10000, lb=0)
drought_model.add_cons_vars(co2tr)

## Imposes that the amount of respired CO2 loaded onto the xylem by the roots rivals (in this case, equals) the export of CO2 to the soil (environment) (equation 15)
co2tr2 = drought_model.problem.Constraint(drought_model.reactions.RXN_E2R_CO2_LIGHT.flux_expression - drought_model.reactions.RXN_R2CP1_CO2_LIGHT.flux_expression, ub=0, lb=0)
co2tr3 = drought_model.problem.Constraint(drought_model.reactions.RXN_E2R_CO2_DARK.flux_expression - drought_model.reactions.RXN_R2CP1_CO2_DARK.flux_expression, ub=0, lb=0)
drought_model.add_cons_vars(co2tr2)  
drought_model.add_cons_vars(co2tr3) 

# limit photon uptake in the light based on calculations described in the paper (equation 16)
drought_model.reactions.RXN_E2L_LIGHT__L___LIGHT.lower_bound = 0;drought_model.reactions.RXN_E2L_LIGHT__L___LIGHT.upper_bound = leaf_photon_uptake_ub

## Allowing storage of respired CO2 in the stem, working toward building equation 17
drought_model.add_boundary(drought_model.metabolites.MET_carbon_dioxide_c__S___DARK,type = "demand")
drought_model.add_boundary(drought_model.metabolites.MET_carbon_dioxide_c__S___LIGHT,type = "sink")

#This prevents CO2 storage during the day
drought_model.reactions.SK_MET_carbon_dioxide_c__S___LIGHT.upper_bound = 0; 

# Imposes that 10% of respired CO2 is stored into the stems during the night (equation 17)
# requiring 10% storage
co2storage = drought_model.problem.Constraint(0.1*drought_model.reactions.RXN_E2S_CO2_DARK.flux_expression - drought_model.reactions.DM_MET_carbon_dioxide_c__S___DARK.flux_expression, ub = 0, lb = 0)

# requiring the storage at night to equal its use in day
co2storage2 = drought_model.problem.Constraint(drought_model.reactions.SK_MET_carbon_dioxide_c__S___LIGHT.flux_expression + drought_model.reactions.DM_MET_carbon_dioxide_c__S___DARK.flux_expression, ub = 0, lb = 0)

drought_model.add_cons_vars(co2storage)
drought_model.add_cons_vars(co2storage2)

# Adding connecting starch reactions for creating equations 18 and 19
## Define sink and demand reactions
## sing reactions are for removal of starch from storage, demand are for adding starch to storage
drought_model.add_boundary(drought_model.metabolites.MET_starch_p__L___LIGHT, type="demand")
drought_model.add_boundary(drought_model.metabolites.MET_starch_p__S___LIGHT, type="demand")
drought_model.add_boundary(drought_model.metabolites.MET_starch_p__L___DARK, type="sink")
drought_model.add_boundary(drought_model.metabolites.MET_starch_p__S___DARK, type="sink")
    
# Only allows consumption of starch at night
drought_model.reactions.SK_MET_starch_p__L___DARK.upper_bound = 0
drought_model.reactions.SK_MET_starch_p__S___DARK.upper_bound = 0

# by default, demand reactions have LB = 0, so don't need to specify this to ensure only storage during day. 

## Requiring that starch producded during the day is used at night, and visa versa (equations 18 and 19)
# leaf
starchprodL = drought_model.problem.Constraint(drought_model.reactions.DM_MET_starch_p__L___LIGHT.flux_expression + drought_model.reactions.SK_MET_starch_p__L___DARK.flux_expression, lb = 0, ub = 0)
drought_model.add_cons_vars(starchprodL)

#stem
starchprodS = drought_model.problem.Constraint(drought_model.reactions.DM_MET_starch_p__S___LIGHT.flux_expression + drought_model.reactions.SK_MET_starch_p__S___DARK.flux_expression, lb = 0, ub = 0)
drought_model.add_cons_vars(starchprodS)

## Imposes a photorespiration rate of 25% during the day (equation 21)
photoresp = drought_model.problem.Constraint(drought_model.reactions.RXN_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p__L___LIGHT.flux_expression + 2*drought_model.reactions.RXN_RXN_961_p__L___LIGHT.flux_expression, ub=0,lb=0)
drought_model.add_cons_vars(photoresp)

## toward eqauations 23
#based on maintaining leaf to shoot ratio, define what is in the parentehses of constraints 23
alpha = (np.log(Init_L_mass) - np.log(Init_L_mass / Init_S_mass) - np.log(Init_S_mass))/171

# Links the growth rates of the stem and leaf, maintaining the L/S ratio as in control plants
LtoSratio = drought_model.problem.Constraint((alpha + drought_model.reactions.RXN_BiomassRxn__L___DARK.flux_expression) - drought_model.reactions.RXN_BiomassRxn__S___DARK.flux_expression, ub = 0, lb = 0)

drought_model.add_cons_vars(LtoSratio)

## Limit growth rates of tissues based on those of control (equation 25)
drought_model.reactions.RXN_BiomassRxn__R___DARK.lower_bound = cont_root_g * r_g_frac_H2 ; drought_model.reactions.RXN_BiomassRxn__R___DARK.upper_bound = cont_root_g * r_g_frac_H2
drought_model.reactions.RXN_BiomassRxn__S___DARK.lower_bound = cont_stem_g * s_g_frac_H2 ; drought_model.reactions.RXN_BiomassRxn__S___DARK.upper_bound = cont_stem_g * s_g_frac_H2
drought_model.reactions.RXN_BiomassRxn__L___DARK.lower_bound = 0 ; drought_model.reactions.RXN_BiomassRxn__L___DARK.upper_bound = cont_leaf_g
drought_model.reactions.RXN_BiomassRxn__L___LIGHT.lower_bound = 0 ; drought_model.reactions.RXN_BiomassRxn__L___LIGHT.upper_bound = 0
drought_model.reactions.RXN_BiomassRxn__S___LIGHT.lower_bound = 0 ; drought_model.reactions.RXN_BiomassRxn__S___LIGHT.upper_bound = 0
drought_model.reactions.RXN_BiomassRxn__R___LIGHT.lower_bound = 0 ; drought_model.reactions.RXN_BiomassRxn__R___LIGHT.upper_bound = 0

# Limit carbon dioxide uptake by  CO2 uptake of control (equation 26)
drought_model.reactions.RXN_E2L_CO2_LIGHT.upper_bound = co2_min_cont * a_frac_H2

In [7]:
alpha

5.1940258461995625e-18

#### set the objective and direction

In [8]:
# Sets the model's objective and objective direction
drought_model.objective = drought_model.reactions.RXN_ATPM_c__S___DARK
drought_model.objective_direction = 'max'

#### Solve the model, report results

In [9]:
# Solves the model
d_solution = cobra.flux_analysis.pfba(drought_model)

In [10]:
# Print statements
print("total sum of fluxes:\t\t"+str(d_solution.objective_value)+" mmol/gDWday")
print("starch production in the leaf:\t"+str(d_solution.fluxes.DM_MET_starch_p__L___LIGHT) +" mmol/gDWday")
print("starch production in the stem:\t"+str(d_solution.fluxes.DM_MET_starch_p__S___LIGHT) +" mmol/gDWday")

print("Day leaf CO2 uptake:\t\t"+ str(d_solution.fluxes.RXN_E2L_CO2_LIGHT)+" mmol/gDWday")
print("Night leaf CO2 efflux:\t\t"+ str(-d_solution.fluxes.RXN_E2L_CO2_DARK)+" mmol/gDWday")
print("Day stem CO2 efflux:\t\t"+ str(d_solution.fluxes.RXN_E2S_CO2_LIGHT)+" mmol/gDWday")
print("Night stem CO2 efflux:\t\t"+ str(d_solution.fluxes.RXN_E2S_CO2_DARK)+" mmol/gDWday")
print("Day root CO2 efflux:\t\t"+ str(d_solution.fluxes.RXN_E2R_CO2_LIGHT)+" mmol/gDWday")
print("Night root CO2 efflux:\t\t"+ str(d_solution.fluxes.RXN_E2R_CO2_DARK)+" mmol/gDWday")

total_efflux = -d_solution.fluxes.RXN_E2L_CO2_DARK + d_solution.fluxes.RXN_E2S_CO2_LIGHT + d_solution.fluxes.RXN_E2S_CO2_DARK + d_solution.fluxes.RXN_E2R_CO2_LIGHT + d_solution.fluxes.RXN_E2R_CO2_DARK
print("Total CO2 efflux:\t\t"+ str(total_efflux)+" mmol/gDWday")

net_CO2 = d_solution.fluxes.RXN_E2L_CO2_LIGHT - total_efflux

print("Net CO2 uptake:\t\t\t"+ str(net_CO2)+" mmol/gDWday\n")

print("Leaf growth rate:\t\t\t"+ str(d_solution.fluxes.RXN_BiomassRxn__L___DARK)+" mmol/gdWday")
print("Stem growth rate (inner obj):\t\t"+ str(d_solution.fluxes.RXN_BiomassRxn__S___DARK)+" mmol/gdWday")
print("Root growth rate:\t\t\t"+ str(d_solution.fluxes.RXN_BiomassRxn__R___DARK)+" mmol/gdWday")
print("Leaf growth rate (% of control):\t"+ str((d_solution.fluxes.RXN_BiomassRxn__L___DARK/cont_leaf_g)*100)+"%")
print("Stem growth rate (% of control):\t"+ str((d_solution.fluxes.RXN_BiomassRxn__S___DARK/cont_stem_g)*100)+"%")
print("Root growth rate (% of control):\t"+ str((d_solution.fluxes.RXN_BiomassRxn__R___DARK/cont_root_g)*100)+"%")

print("\nEstimated NGAM:\t\t",d_solution.fluxes.RXN_ATPM_c__S___DARK)

gam_cost = 693.12 * (d_solution.fluxes.RXN_BiomassRxn__L___DARK + d_solution.fluxes.RXN_BiomassRxn__S___DARK + d_solution.fluxes.RXN_BiomassRxn__R___DARK)

print("\nGAM cost:\t\t\t"+ str(gam_cost)+" mmol/gDWday")

df = pd.DataFrame([[d_solution.fluxes[n]] for n in range(0, len(d_solution.fluxes))])
df.index = [rxn.id for rxn in drought_model.reactions]
df.to_csv('pFBA_results_drought_H2.csv', sep=',', index=True)

total sum of fluxes:		3499.0090955778146 mmol/gDWday
starch production in the leaf:	0.5612319413948457 mmol/gDWday
starch production in the stem:	0.28296118630309813 mmol/gDWday
Day leaf CO2 uptake:		18.72 mmol/gDWday
Night leaf CO2 efflux:		-0.0 mmol/gDWday
Day stem CO2 efflux:		0.0 mmol/gDWday
Night stem CO2 efflux:		9.209157836038795 mmol/gDWday
Day root CO2 efflux:		2.4184419751734376 mmol/gDWday
Night root CO2 efflux:		2.4424737514799717 mmol/gDWday
Total CO2 efflux:		14.070073562692203 mmol/gDWday
Net CO2 uptake:			4.649926437307796 mmol/gDWday

Leaf growth rate:			0.0001422273999999948 mmol/gdWday
Stem growth rate (inner obj):		0.0001422274 mmol/gdWday
Root growth rate:			0.00275366769 mmol/gdWday
Leaf growth rate (% of control):	7.307368230683123%
Stem growth rate (% of control):	4.0%
Root growth rate (% of control):	93.0%

Estimated NGAM:		 19.347535801387505

GAM cost:			2.105783460268796 mmol/gDWday


#### Write growth rates to an output file

In [11]:
growth_out = open("drought_g_rates_H2.txt","w")
growth_out.write("leaf: "+str(d_solution.fluxes.RXN_BiomassRxn__L___DARK))
growth_out.write("\nstem: "+str(d_solution.fluxes.RXN_BiomassRxn__S___DARK))
growth_out.write("\nroot: "+str(d_solution.fluxes.RXN_BiomassRxn__R___DARK))
growth_out.close()

#### Report NGAM estimate

In [12]:
ngam_out = open("iPotri2999D_NGAM_H2_est.txt","w")
ngam_out.write(str(d_solution.fluxes.RXN_ATPM_c__S___DARK))
ngam_out.close()